In [ ]:
from collections import defaultdict

import os
import warnings
from pathlib import Path

import pandas as pd
import patsy
import numpy as np
import seaborn as sns
import time

import matplotlib.pyplot as plt
import sklearn.manifold as skmf
import sklearn.decomposition as skdc
import sklearn.metrics as skmr

import condo

from combat import combat

In [ ]:
this_file = os.path.realpath('__file__')
data_path = os.path.join(Path(this_file).parent.parent, 'data')
data_path = '../data/'
all_pheno = pd.read_csv(os.path.join(data_path, 'pheno25.csv'), index_col=0)
all_expr = pd.read_csv(os.path.join(data_path, 'expr25.csv'), index_col=0)

num_random = 10
mmd_size = 20
sil_results = defaultdict(list)
sil_batches = defaultdict(list)
ch_results = defaultdict(list)
ch_batches = defaultdict(list)
db_results = defaultdict(list)
db_batches = defaultdict(list)

num_removecancer = 7

In [ ]:
for rix in range(num_random):
    print(f"rix:{rix} {num_removecancer}")
    rng = np.random.RandomState(rix)
    cancer2_ixs = np.where((all_pheno.batch == 2) & (all_pheno.result == "Cancer"))[0]
    victim_ixs = list(rng.choice(cancer2_ixs, size=num_removecancer, replace=False))
    chosen_ixs = [ix for ix in range(all_pheno.shape[0]) if ix not in victim_ixs]
    pheno = all_pheno.iloc[chosen_ixs, :]
    expr = all_expr.iloc[chosen_ixs, :]
    exprTranspose = expr.T

    # Combat
    start_time = time.time()
    mod = patsy.dmatrix("~ age + cancer", pheno, return_type="dataframe")
    exprTranspose_combat = combat(exprTranspose, pheno['batch'], mod, "age")
    expr_combat = exprTranspose_combat.T
    duration_combat = time.time() - start_time
    #print(f"Combat time: {duration_combat}")
    
    # Gaussian OT
    start_time = time.time()
    lder = condo.AdapterGaussianOT(
        transform_type="location-scale",
    )
    lder.fit(
        expr[pheno.batch == 2].values, 
        expr[pheno.batch == 5].values,
    )
    expr_linear = lder.transform(expr.values)
    duration_linear = time.time() - start_time
    #print(f"Gaussian OT time: {duration_linear}")
    expr_linear[np.where(pheno.batch == 5)[0],:] = expr.values[np.where(pheno.batch == 5)[0],:]

    # MMD
    start_time = time.time()
    mmder = condo.AdapterMMD(
        transform_type="location-scale",
        n_epochs=100,
        learning_rate=1e-2,
        mmd_size=mmd_size,
        verbose=False,
    )
    mmder.fit(
        expr[pheno.batch == 2].values, 
        expr[pheno.batch == 5].values,
    )
    expr_mmd = mmder.transform(expr.values)
    duration_mmd = time.time() - start_time
    #print(f"MMD time: {duration_mmd}")
    expr_mmd[np.where(pheno.batch == 5)[0],:] = expr.values[np.where(pheno.batch == 5)[0],:]
    
    # Condo Linear ReverseKL
    start_time = time.time()
    cder_clinear = condo.ConDoAdapterKLD(
        transform_type="location-scale",
        verbose=0,
    )
    cder_clinear.fit(
        expr[pheno.batch == 2].values, 
        expr[pheno.batch == 5].values,
        pheno[pheno.batch==2].result.values.reshape(-1, 1).astype(str),
        pheno[pheno.batch==5].result.values.reshape(-1, 1).astype(str),
    )
    expr_clinear = cder_clinear.transform(expr.values)
    duration_clinear = time.time() - start_time
    #print(f"ConDo Linear-ReverseKL time: {duration_clinear}")
    expr_clinear[np.where(pheno.batch == 5)[0],:] = expr.values[np.where(pheno.batch == 5)[0],:]

    start_time = time.time()
    cder_mmd = condo.ConDoAdapterMMD(
        transform_type="location-scale",
        n_epochs=100,
        learning_rate=1e-2,
        mmd_size=mmd_size,
        verbose=False,
    )
    cder_mmd.fit(
        expr[pheno.batch == 2].values, 
        expr[pheno.batch == 5].values,
        pheno[pheno.batch==2].result.values.reshape(-1, 1).astype(str),
        pheno[pheno.batch==5].result.values.reshape(-1, 1).astype(str),
    )
    expr_cmmd = cder_mmd.transform(expr.values)
    duration_cmmd = time.time() - start_time
    #print(f"ConDo MMD time: {duration_cmmd}")
    expr_cmmd[np.where(pheno.batch == 5)[0],:] = expr.values[np.where(pheno.batch == 5)[0],:]
    
    dinfos = [
        (0, "Original", expr),
        (1, "Combat", expr_combat),
        (1, "Gaussian OT", expr_linear),
        (2, "MMD", expr_mmd),
        (3, "ConDo Gaussian KLD", expr_clinear),
        (4, "ConDo MMD", expr_cmmd),
    ]
    for dix, dname, dset in dinfos:
        ixs = rng.choice(dset.shape[1], size=10000, 
        sil_result = skmr.silhouette_score(dset, pheno.result, metric='euclidean')
        sil_batch = skmr.silhouette_score(dset, pheno.batch, metric='euclidean')
        sil_results[dname].append(sil_result)
        sil_batches[dname].append(sil_batch)
        ch_result = skmr.calinski_harabasz_score(dset, pheno.result)
        ch_batch = skmr.calinski_harabasz_score(dset, pheno.batch)
        ch_results[dname].append(ch_result)
        ch_batches[dname].append(ch_batch)
        db_result = skmr.davies_bouldin_score(dset, pheno.result)
        db_batch = skmr.davies_bouldin_score(dset, pheno.batch)
        db_results[dname].append(db_result)
        db_batches[dname].append(db_batch)

In [ ]:
print("batches")
for key in ch_batches:
    print(key, np.mean(ch_batches[key]), np.std(ch_batches[key]))
print("results")
for key in ch_results:
    print(key, np.mean(ch_results[key]), np.std(ch_results[key]))

In [ ]:
print("batches")
for key in sil_batches:
    #print(key, np.mean(sil_batches[key]), np.std(sil_batches[key]))
    dtitle = f"{key} {np.mean(sil_batches[key]):.4f} ({np.std(sil_batches[key])/num_random:.4f})"
    print(dtitle)
print("results")
for key in sil_results:
    #print(key, np.mean(sil_results[key]), np.std(sil_results[key]))
    dtitle = f"{key} {np.mean(sil_results[key]):.4f} ({np.std(sil_results[key])/num_random:.4f})"
    print(dtitle)

In [ ]:
dset.shape